**Chapter 3 - Intro to Page Layout**

The ReportLab engineers describe PLATYPUS as having several layers (from highest to lowest level):
- **DocTemplates** - the outermost container of your page
- **PageTemplates** - specifies the layout of your page
- **Frames** - kind of like a sizer in a desktop user interface.
Basically it provides a region that contains other flowables
- **Flowables** - A text or graphic element that can be “flowed”
across page boundaries, such as a paragraph of text. This does not
include footers and headers.
- **pdfgen.Canvas** - The lowest level of ReportLab, and one that we have
already covered. It will actually receive its instructions
from one or more of the upper layers and “paint” your
document accordingly.

In [ ]:
# form_letter.py

import time
from reportlab.lib.enums import TA_JUSTIFY                                  #'Text Alignment - Justify' - value of alignment corresponding with justified text
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image  #all necessary for Platypus usage: Paragraph, Spacer, Image are all flowables
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle        #ParagraphStyle: by importing, can now create a new style and add to styles list
from reportlab.lib.units import inch

def form_letter():

    doc = SimpleDocTemplate("form_letter.pdf",
                            pagesize=letter,
                            rightMargin=72,                         #as always, these values are in points (not inches/pixels/whatever)
                            leftMargin=72,
                            topMargin=72,
                            bottomMargin=18)
    flowables = []                                                  #set up list of flowables
    logo = r'''C:\Users\lucy\OneDrive - Aldatu Biosciences
               \Desktop\PANDAA qPCR Results\vhf\aldatulogo_icon.gif'''
    magName = "Pythonista"
    issueNum = 12
    subPrice = "99.00"
    limitedDate = "03/05/2010"
    freeGift = "tin foil hat"

    formatted_time = time.ctime()                                   #get current time, formatted: Fri Jan 3 11:13:16 2025
    full_name = "Mike Driscoll"
    address_parts = ["411 State St.", "Waterloo, IA 50158"]

    im = Image(logo, 1.5*inch, 1.5*inch)                                #create im flowable by passing filepath and size info (default size is equal to original image size)
    flowables.append(im)

    styles = getSampleStyleSheet()                                  #styles variable contains all available platypus styles
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY)) #add new paragraph style called Justify
    
    ptext = '<font size=12>%s</font>' % formatted_time
    flowables.append(Paragraph(ptext, styles["Normal"]))            #'Normal' style definition uses _baseFontName size 10 - pretty much the same as our new 'Justify,' but aligned left instead of justified
    flowables.append(Spacer(1, 12))

    # Create return address
    ptext = '<font size=12>%s</font>' % full_name
    flowables.append(Paragraph(ptext, styles["Normal"]))
    for part in address_parts:
        ptext = '<font size=12>%s</font>' % part.strip()
        flowables.append(Paragraph(ptext, styles["Normal"]))
    flowables.append(Spacer(1, 12))

    ptext = '<font size=12>Dear %s:</font>' % full_name.split()[0].strip()
    flowables.append(Paragraph(ptext, styles["Normal"]))
    flowables.append(Spacer(1, 12))

    ptext = '''
    <font size=12>We would like to welcome you to our subscriber
    base for {magName} Magazine! You will receive {issueNum} issues at
    the excellent introductory price of ${subPrice}. Please respond by
    {limitedDate} to start receiving your subscription and get the
    following free gift: {freeGift}.</font>
    '''.format(magName=magName,
               issueNum=issueNum,
               subPrice=subPrice,
               limitedDate=limitedDate,
               freeGift=freeGift)
    flowables.append(Paragraph(ptext, styles["Justify"]))           #keep appending flowables to our list
    flowables.append(Spacer(1, 12))

    ptext = '''<font size=12>Thank you very much and we look
    forward to serving you.</font>'''
    flowables.append(Paragraph(ptext, styles["Justify"]))
    flowables.append(Spacer(1, 12))
    ptext = '<font size=12>Sincerely,</font>'
    flowables.append(Paragraph(ptext, styles["Normal"]))
    flowables.append(Spacer(1, 48))
    ptext = '<font size=12>Ima Sucker</font>'
    flowables.append(Paragraph(ptext, styles["Normal"]))
    flowables.append(Spacer(1, 12))

    doc.build(flowables)                                            #using list we've created, build document

if __name__ == '__main__':
    form_letter()

In [3]:
# doc_build_different_first_and_subsequent_pages.py

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.rl_config import defaultPageSize
from reportlab.lib.units import inch


def first_page(canvas, document):                                       #function to add centered title to page
    
    title = 'PLATYPUS Demo'
    PAGE_HEIGHT = defaultPageSize[1]
    PAGE_WIDTH = defaultPageSize[0]

    canvas.saveState()                                                  
    canvas.setFont('Times-Bold', 18)
    canvas.drawCentredString(PAGE_WIDTH/2.0, PAGE_HEIGHT-108, title)    #draw title

    canvas.setFont('Times-Roman', 10)
    text = 'Welcome to the first PLATYPUS Page!'
    canvas.drawString(inch, 10*inch, text)                              #draw first line
    canvas.restoreState()


def later_pages(canvas, document):                                      #function to add page number to lower right corner

    canvas.saveState()
    canvas.setFont('Helvetica', 10)
    canvas.drawString(7*inch, 0.5*inch,
                      'Page {}'.format(document.page))                  #add page number
    canvas.restoreState()


def create_document():                                                  #function to create document content

    doc = SimpleDocTemplate("platypus_first_later.pdf",
                            pagesize=letter,
                            rightMargin=72,
                            leftMargin=72,
                            topMargin=72,
                            bottomMargin=18)
    styles = getSampleStyleSheet()
    flowables = []
    spacer = Spacer(1, 0.25*inch)

    # Create a lot of content to make a multipage PDF
    for i in range(50):
        text = 'Paragraph #{}'.format(i)
        para = Paragraph(text, styles["Normal"])
        flowables.append(para)
        flowables.append(spacer)

    doc.build(flowables, onFirstPage=first_page, onLaterPages=later_pages) #build: use list of flowables as content, add centered title to first page, add page numbers to subsequent pages


if __name__ == '__main__':
    create_document()

**Flowable Methods**

We’ve been talking about flowables since the beginning of this chapter and it’s
high time we actually define what a flowable is in ReportLab. A Flowable is
something that can be drawn and which has the following
methods:
- draw (access using Flowable.drawOn()) - draws flowable directly on canvas
- wrap (access using Flowable.wrapOn()) - wraps text (/flowables) using available width/height, accomodating frame boundaries
- split (optionally) - splits text/flowables using available width/height, accomodating frame boundaries

If you need to find out what your vertical spacing is, then you can call
**getSpaceAfter** or **getSpaceBefore** to find out. These methods will tell you how
much space should follow or precede a flowable. Also all flowables have an
**hAlign** property: (‘LEFT’, ‘RIGHT’, ‘CENTER’ or ‘CENTRE’). Paragraphs
ignore this property, but tables, images and other objects which are smaller than
the width of the frame will use this
property to determine their horizontal position.

In [4]:
# draw_wrap_flowable.py

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

def mixed():

    my_canvas = canvas.Canvas("mixed_flowables.pdf", pagesize=letter)
    styles = getSampleStyleSheet()
    width, height = letter

    text = "Hello, I'm a Paragraph"
    para = Paragraph(text, style=styles["Normal"])
    para.wrapOn(my_canvas, width, height)           #wrapOn is mostly useful with longer paragraphs - wraps text based on available width/height
    para.drawOn(my_canvas, 20, 760)                 #using drawOn, we can avoid doc.build() altogether and draw flowable directly on canvas
    
    # NB: split is basically the opposite of wrap - using available width/height, split will split the flowable appropriately, should the flowable reach a frame boundary

    my_canvas.save()

if __name__ == '__main__':
    mixed()

**Templates**

We have been using the **SimpleDocTemplate** up to this point for creating our
multi-page PDFs. However the **SimpleDocTemplate** is actually a subclass of
**BaseDocTemplate**, which implements the magic behind your document’s
formatting. When you create an instance of this class, it will contain one or more
other **PageTemplates** that are a description of the layout of each of the pages of
your document. When you call the **build** method of a template class, it will
process the list of **Flowables** that we’ve been creating to create your PDF
document.

In [5]:
# two_column_frame_doctemplate_demo.py

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.pdfgen.canvas import Canvas
from reportlab.platypus import Paragraph, Frame, BaseDocTemplate

def frame_demo():

    my_canvas = Canvas("frame_demo.pdf",
                        pagesize=letter)
    styles = getSampleStyleSheet()
    normal = styles['Normal']       #extract some styles for later use - normal, heading
    heading = styles['Heading1']

    flowables = []
    flowables.append(Paragraph('Heading #1', heading))
    flowables.append(Paragraph('Paragraph #1', normal))

    right_flowables = []
    right_flowables.append(Paragraph('Heading #2', heading))
    right_flowables.append(Paragraph('ipsum lorem', normal))

    left_frame = Frame(inch, inch, width=3*inch, height=9*inch, showBoundary=1)
    right_frame = Frame(4*inch, inch, width=3*inch, height=9*inch)                  #these frames are the same size, but right frame is 4 inches to the right

    left_frame.addFromList(flowables, my_canvas)                                    
    right_frame.addFromList(right_flowables, my_canvas)

    # NB: Note that you don’t usually need to use a Frame directly as other Flowables will do that work for you, unless you want to create a custom PageTemplate or other custom Flowable.

    my_canvas.save()

def build_demo():
    doc = BaseDocTemplate('filename.pdf',       #only necessary when creating instance of class
                          pagesize=defaultPageSize,
                          pageTemplates=[],
                          showBoundary=0,
                          leftMargin=inch,
                          rightMargin=inch,
                          topMargin=inch,
                          bottomMargin=inch,
                          allowSplitting=1,     #tells ReportLab that it should try to split each Flowable across the Frames
                          title=None,
                          author=None,
                          _pageBreakQuick=1,
                          encrypt=None)

if __name__ == '__main__':
    frame_demo()

In [6]:
# flowable_orientation.py

from reportlab.lib.pagesizes import letter, landscape
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, PageBreak
from reportlab.platypus import Frame, PageTemplate, NextPageTemplate, Spacer


def alternate_orientations():

    doc = SimpleDocTemplate("orientations.pdf",
                            pagesize=letter,
                            rightMargin=72,
                            leftMargin=72,
                            topMargin=72,
                            bottomMargin=18)
    styles = getSampleStyleSheet()
    normal = styles["Normal"]

    margin = 0.5 * inch
    frame = Frame(margin, margin, doc.width, doc.height,
                  id='frame')
    portrait_template = PageTemplate(id='portrait',
                                     frames=[frame],
                                     pagesize=letter)
    landscape_template = PageTemplate(id='landscape',
                                      frames=[frame],
                                      pagesize=landscape(letter))
    doc.addPageTemplates([portrait_template, landscape_template])               #add custom page templates to our document

    story = []                                                                  #empty list to add flowables to
    story.append(Paragraph('This is a page in portrait orientation', normal))

    # Change to landscape orientation
    story.append(NextPageTemplate('landscape'))                                 #NextPageTemplate tells ReportLab what template to start using on the next page
    story.append(PageBreak())                                                   #PageBreak flowable inserts page break into doc
    story.append(Spacer(inch, 2*inch))                                          #Spacer flowable adds carriage return / space between flowables - needed here because landscape page is shorter in height
    story.append(Paragraph('This is a page in landscape orientation', normal))

    # Change back to portrait
    story.append(NextPageTemplate('portrait'))                                  
    story.append(PageBreak())
    story.append(Paragraph("Now we're back in portrait mode again", normal))
    doc.build(story)

if __name__ == '__main__':
    alternate_orientations()